# Langchain basics

In [ ]:
pip install -r requirements.txt

## Python-dotenv

In [95]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

### LLM Models(Wrappers): GPT-3

In [21]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


In [22]:
output = llm('explain quantum mechanics in one sentence')
print(output)



Quantum mechanics is a physical theory that describes the behavior of matter and energy on the atomic and subatomic level.


In [23]:
print(llm.get_num_tokens("explain quantum mechanics in one sentence"))

7


In [37]:
output =  llm.generate(["... is the capital of France.", "What is the formula for the area of circle?"])


In [38]:
print(output.generations)

[[Generation(text='\n\nParis.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe formula for the area of a circle is A = πr², where r is the radius of the circle.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [40]:
# Only one answer - 2nd question
print(output.generations[1][0].text)



The formula for the area of a circle is A = πr², where r is the radius of the circle.


In [41]:
# Only one answer - 1st question
print(output.generations[0][0].text)



Paris.


In [42]:
# number of generations
len(output.generations)

2

In [49]:
output =  llm.generate(["Write an original taglines for burger restaurant"] * 3)

In [50]:
print(output)

generations=[[Generation(text='\n\n"A Taste of Burgers That Will Leave You Hungry for More!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Burgers that will make your taste buds smile!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Taste the Juiciest Burgers in Town!"', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'completion_tokens': 44, 'total_tokens': 68, 'prompt_tokens': 24}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('439795b1-af4a-4aac-b633-d0b086e0ffb5')), RunInfo(run_id=UUID('8682121d-5190-4d8e-8a58-83d9e17c3659')), RunInfo(run_id=UUID('cc125b94-477b-4c87-b7ef-a515a480bcdb'))]


In [51]:
for i in output.generations:
    print(i[0].text)



"A Taste of Burgers That Will Leave You Hungry for More!"


"Burgers that will make your taste buds smile!"


"Taste the Juiciest Burgers in Town!"


## ChatModels: GPT-3.5-Turbo and GPT-4

In [53]:
# import schema for the structure called messages
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [57]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content="You are a physicist and respond only in German."),
    HumanMessage(content="Explain quantum mechanics in one sentence")
]
output = chat(messages)

In [58]:
print(output)

content='Quantenmechanik beschreibt das Verhalten von Teilchen auf der subatomaren Ebene und ermöglicht es uns, ihre Eigenschaften und Wechselwirkungen zu verstehen.'


In [59]:
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf der subatomaren Ebene und ermöglicht es uns, ihre Eigenschaften und Wechselwirkungen zu verstehen.


## Prompt Template
creates dynamic prompts for LLMs

In [69]:
from langchain import PromptTemplate

In [70]:
template = """You are an experienced virologist.
Write a few sentences about the following {virus} in {language}"""

prompt = PromptTemplate(
    input_variables = ["virus", "language"],
    template = template
)
print(prompt)

input_variables=['language', 'virus'] template='You are an experienced virologist.\nWrite a few sentences about the following {virus} in {language}'


In [73]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003", temperature=0.7)
output = llm(prompt.format(virus='ebola', language='french'))
print(output)



Le virus Ebola est un virus mortel et hautement contagieux qui peut infecter les humains et les animaux. Il a été détecté pour la première fois en Afrique de l'Ouest en 2014 et a provoqué une épidémie qui a duré plusieurs années. Les symptômes de cette infection comprennent des fièvres, des maux de tête, des douleurs musculaires et des saignements. Le traitement est difficile et le taux de mortalité est élevé.


## Simple Chains
Chains allow us to combine multiple components together to create a simple application

In [76]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
template = """You are an experienced virologist.
Write a few sentences about the following {virus} in {language}"""


prompt = PromptTemplate(
    input_variables = ["virus", "language"],
    template = template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus': "HSV", 'language': 'french'})

In [77]:
print(output)

L'HSV, ou virus de l'herpès simplex, est un virus à ADN appartenant à la famille des herpèsviridés. Il existe deux types d'HSV : le HSV-1, qui provoque principalement des infections buccales et labiales, et le HSV-2, qui est principalement responsable des infections génitales. Ces virus sont hautement contagieux et se transmettent par contact direct avec les lésions cutanées ou muqueuses. Les symptômes courants de l'infection à HSV incluent des lésions vésiculaires douloureuses, des démangeaisons et des brûlures. Bien qu'il n'existe pas de traitement curatif pour l'HSV, des médicaments antiviraux peuvent aider à réduire la fréquence et la gravité des poussées.


## Sequential Chains
Make a series of calls to one or more LLMs. You can take the output from the chain and use it as input to another chain.
There are 2 types of sequential chains:
1. SimpleSequential Chains
2. General form of Sequential chains

Simple sequential chains represent a series of chains, where each individual chain has a single input and a single output, and the output of one step is used as input to the next


In [79]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables = ["concept"],
    template = """You are an experienced scientist and python developer.
    Write a function that implements the concept of {concept}"""
)
chain1 = LLMChain(llm=llm, prompt=prompt1)

llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables = ["function"],
    template = """Given the python function {function}, describe it as detailed as possible"""
)
chain2 = LLMChain(llm=llm, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose= True)
output = overall_chain.run('linear_regression')




> Entering new SimpleSequentialChain chain...


def linear_regression(x_values, y_values):
    """
    Calculates a linear regression equation based on two lists of 
    x and y values.
    
    Parameters:
    x_values (list) : List of x values
    y_values (list) : List of y values
    
    Returns:
    m (float) : Slope of the linear regression equation
    b (float) : Y-intercept of the linear regression equation
    """
    
    # Calculate the mean of the x and y values
    x_mean = sum(x_values) / len(x_values)
    y_mean = sum(y_values) / len(y_values)
    
    # Calculate the slope of the linear regression equation
    numerator = 0
    denominator = 0
    for i in range(len(x_values)):
        numerator += (x_values[i] - x_mean) * (y_values[i] - y_mean)
        denominator += (x_values[i] - x_mean) ** 2
    m = numerator / denominator
    
    # Calculate the y-intercept of the linear regression equation
    b = y_mean - m * x_mean
    
    return m, b


This function calcu

## LangChain Agents
Agents are enabling tools for LLMs, then they can do calculations, write code, search the webs or run SQL queries

In [ ]:
pip install langchain_experimental

In [92]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.llms import OpenAI

In [94]:
llm = OpenAI(temperature=0)
# Tools are essentially functions agents can use to interact with outside world.
# It can range from genrtal utilities such as maths calculator, search functions to other chains or other agents
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(), #Tools
    verbose=True)
agent_executor.run('Calculate the square root of the factorial of 20 and display it with 4 decimal points')



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I need to calculate the factorial of 20 and then take the square root of that
Action: Python_REPL
Action Input: from math import factorial; print(round(factorial(20)**0.5, 4))
Observation: 1559776268.6285

Thought: I now know the final answer
Final Answer: 1559776268.6285

> Finished chain.


'1559776268.6285'